In [2]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

In [113]:
data = pd.read_csv('bangkok-air-quality.csv')
df = pd.DataFrame(data)
df = df[['date',' pm25',' pm10']]
df.size

8853

This data includes 8,853 blocks

Let's see the overview

In [114]:
df.head()

,date,pm25,pm10
0,2022/2/1,68,31
1,2022/2/2,58,35
2,2022/2/3,66,28
3,2022/1/3,82,51
4,2022/1/4,101,59


In [115]:
df.describe(include='all')

,date,pm25,pm10
count,2951,2951,2951
unique,2951,155,92
top,2022/2/1,,25
freq,1,944,120


### Data cleaning

- Delete the row with missing data
- Include only the data from year 2017-2021

In [116]:
for x in df.index:
    if df.loc[x, " pm25"] == " ":
        df.drop(x, inplace = True)
for x in df.index:
    if df.loc[x, " pm10"] == " ":
        df.drop(x, inplace = True)
for x in df.index:        
    YMD = df.loc[x, "date"].split("/")  # use only data since 2017 - 2021
    if int(YMD[0]) < 2017 or int(YMD[0]) > 2021: 
        df.drop(x, inplace = True)
        
        
df[' pm25'] = df[' pm25'].astype(int)
df[' pm10'] = df[' pm10'].astype(int)

In [117]:
df.describe(include='all')

,date,pm25,pm10
count,1812,1812.000000,1812.000000
unique,1812,NaN,NaN
top,2021/10/1,NaN,NaN
freq,1,NaN,NaN
mean,NaN,82.665011,39.746689
std,NaN,31.912149,16.347191
min,NaN,18.000000,9.000000
25%,NaN,59.000000,27.000000
50%,NaN,74.500000,36.000000
75%,NaN,101.000000,50.000000


In [118]:
df.head()

,date,pm25,pm10
33,2021/10/1,58,24
34,2021/10/2,57,36
35,2021/10/3,72,33
36,2021/10/4,73,43
37,2021/10/5,83,26


In [119]:
df.tail()

,date,pm25,pm10
1847,2017/3/27,91,43
1848,2017/3/28,106,40
1849,2017/3/29,100,37
1850,2017/3/30,88,19
1851,2017/3/31,67,29


Now, we clean the blank value in pm2.5 and pm10 out and eliminate all the data before 2017 and after 2021 out. 

The usable information contains only 1812 rows.

Therefore, it dropped 2951 - 1812 = 1139 rows.

## Try 1 : weekday and weeknumber as inputs

In [24]:
def ToWeekday(date):
    YMD = date.split("/")
    Dayinweek = datetime.datetime(int(YMD[0]), int(YMD[1]), int(YMD[2])).weekday()
    return Dayinweek

def ToWeeknum(date):
    YMD = date.split("/")
    Weeknum = datetime.date(int(YMD[0]), int(YMD[1]), int(YMD[2])).isocalendar().week
    return Weeknum

ToWeekday(df.iloc[0,0])

4

Weekday = 0 = Monday, 6 = Sunday

In [120]:
df['weekday'] = df.apply(lambda row: ToWeekday(row.date), axis = 1)
df['weeknum'] = df.apply(lambda row: ToWeeknum(row.date), axis = 1)
df

,date,pm25,pm10,weekday,weeknum
33,2021/10/1,58,24,4,39
34,2021/10/2,57,36,5,39
35,2021/10/3,72,33,6,39
36,2021/10/4,73,43,0,40
37,2021/10/5,83,26,1,40
...,...,...,...,...,...
1847,2017/3/27,91,43,0,13
1848,2017/3/28,106,40,1,13
1849,2017/3/29,100,37,2,13
1850,2017/3/30,88,19,3,13


Add new columns of each row's weekday and weeknum

### Make default model

In [121]:
X = df.iloc[:, 3:5]
Y = df.iloc[:, 1:3]

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,random_state = 1)
len(Xtrain)

1359

In [122]:
len(Xtest) # 25%

453

In [123]:
mlp_rg = MLPRegressor(max_iter=2000)
mlp_rg.fit(Xtrain, Ytrain)

MLPRegressor(max_iter=2000)

In [124]:
Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

Test R^2 Score : 0.473
Training R^2 Score : 0.504


## Try 2 : day and month as inputs

In [125]:
def ToDay(date):
    YMD = date.split("/")
    return int(YMD[2])

def ToMonth(date):
    YMD = date.split("/")
    return int(YMD[1])
    
df['day'] = df.apply(lambda row: ToDay(row.date), axis = 1)
df['month'] = df.apply(lambda row: ToMonth(row.date), axis = 1)
df

,date,pm25,pm10,weekday,weeknum,day,month
33,2021/10/1,58,24,4,39,1,10
34,2021/10/2,57,36,5,39,2,10
35,2021/10/3,72,33,6,39,3,10
36,2021/10/4,73,43,0,40,4,10
37,2021/10/5,83,26,1,40,5,10
...,...,...,...,...,...,...,...
1847,2017/3/27,91,43,0,13,27,3
1848,2017/3/28,106,40,1,13,28,3
1849,2017/3/29,100,37,2,13,29,3
1850,2017/3/30,88,19,3,13,30,3


In [126]:
X2 = df.iloc[:, 5:7]
Y2 = df.iloc[:, 1:3]

Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X2,Y2,random_state = 1)

In [128]:
mlp_rg2 = MLPRegressor(max_iter=2000)
mlp_rg2.fit(Xtrain2, Ytrain2)

MLPRegressor(max_iter=2000)

In [129]:
Ypred2 = mlp_rg2.predict(Xtest2)

print('Test R^2 Score : %.3f'%mlp_rg2.score(Xtest2, Ytest2)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg2.score(Xtrain2, Ytrain2))

Test R^2 Score : 0.450
Training R^2 Score : 0.482


### Using weekday and week number is slightly better than using day and month

# --- Optimization by changing hyperparameter
## Focus on try 1 -- which we got better result
### - epoch & solver
- The solvers tried **didn't include lbfgs because** when I tried, even max_iter = 5,000 didn't enough to be converaged in the optimization (warned by the program)

In [130]:
parameter_space = {
    'max_iter': np.arange(2000,4001,200),
    'solver': ['adam', 'sgd'], # lbfgs is too expensive and problematic (even 5,000 epoch isn't enough)
}

clf = GridSearchCV(mlp_rg, parameter_space, n_jobs=-1, cv=5)
clf.fit(Xtrain, Ytrain)

GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=2000), n_jobs=-1,
             param_grid={'max_iter': array([2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000]),
                         'solver': ['adam', 'sgd']})

In [131]:
print("Best epoch and best solver : ", clf.best_params_)

Best epoch and best solver :  {'max_iter': 2200, 'solver': 'adam'}


So, best epoch to use is 2,200 and we shouldn't use epoch below that because program warns that it's not enough for the optimization.

For the solver for weight optimization, adam is best.

In [132]:
mlp_rg = MLPRegressor(max_iter=2200)
mlp_rg.fit(Xtrain, Ytrain)
Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

Test R^2 Score : 0.484
Training R^2 Score : 0.505


From changing 2,000 epochs to 2,200 epochs, R^2 increases from 0.473 to 0.484

### - Activation Function & learning rate

In [133]:
parameter_space = {
    'activation': ['relu', 'tanh', 'logistic', 'identity'],
    'learning_rate' : ['constant', 'invscaling', 'adaptive']
}

clf = GridSearchCV(mlp_rg, parameter_space, n_jobs=-1, cv=5)
clf.fit(Xtrain, Ytrain)

GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=2200), n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh', 'logistic', 'identity'],
                         'learning_rate': ['constant', 'invscaling',
                                           'adaptive']})

In [134]:
print(clf.best_params_)

{'activation': 'relu', 'learning_rate': 'constant'}


So, the hidden layer activation function and learning rate are optimized at default

### Therefore, epoch = 2,200 | solver, activation, and learning rate are optimized at default

## Try 2

### - epoch and solver

In [147]:
parameter_space = {
    'max_iter': np.arange(2000,4001,200),
    'solver': ['adam', 'sgd'], # lbfgs is too expensive and problematic (even 5,000 epoch isn't enough)
}

clf = GridSearchCV(mlp_rg2, parameter_space, n_jobs=-1, cv=5)
clf.fit(Xtrain2, Ytrain2)

GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=2600), n_jobs=-1,
             param_grid={'max_iter': array([2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000]),
                         'solver': ['adam', 'sgd']})

In [148]:
print(clf.best_params_)

{'max_iter': 3800, 'solver': 'adam'}


In [149]:
mlp_rg2 = MLPRegressor(max_iter=3800)
mlp_rg2.fit(Xtrain2, Ytrain2)
Ypred2 = mlp_rg2.predict(Xtest2)

print('Test R^2 Score : %.3f'%mlp_rg2.score(Xtest2, Ytest2)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg2.score(Xtrain2, Ytrain2))

Test R^2 Score : 0.473
Training R^2 Score : 0.504


So, best epoch for try 2 is 2,600

It improves R^2 from 0.450 to 0.473

### - Activation function

In [153]:
parameter_space = {
    'activation': ['relu', 'tanh'] # because trying 4 functions at a time and the optimization hasn't converged
}

clf = GridSearchCV(mlp_rg2, parameter_space, n_jobs=-1, cv=5)
clf.fit(Xtrain2, Ytrain2)

/Users/stevejen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3800) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=3800), n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh']})

In [154]:
print(clf.best_params_)

{'activation': 'relu'}


In [155]:
parameter_space = {
    'activation': ['relu', 'logistic', 'identity']
}

clf = GridSearchCV(mlp_rg2, parameter_space, n_jobs=-1, cv=5)
clf.fit(Xtrain2, Ytrain2)

/Users/stevejen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3800) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/stevejen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3800) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=3800), n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic', 'identity']})

In [156]:
print(clf.best_params_)

{'activation': 'relu'}


The best activation function for model 2 is still relu

### - Learning rate

In [157]:
parameter_space = {
    'learning_rate' : ['constant', 'invscaling', 'adaptive']
}

clf = GridSearchCV(mlp_rg2, parameter_space, n_jobs=-1, cv=5)
clf.fit(Xtrain2, Ytrain2)

GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=3800), n_jobs=-1,
             param_grid={'learning_rate': ['constant', 'invscaling',
                                           'adaptive']})

In [158]:
print(clf.best_params_)

{'learning_rate': 'invscaling'}


In [159]:
mlp_rg2 = MLPRegressor(max_iter=3800, learning_rate='invscaling')
mlp_rg2.fit(Xtrain2, Ytrain2)
Ypred2 = mlp_rg2.predict(Xtest2)

print('Test R^2 Score : %.3f'%mlp_rg2.score(Xtest2, Ytest2)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg2.score(Xtrain2, Ytrain2))

Test R^2 Score : 0.475
Training R^2 Score : 0.504


Changing learning rate from constant to invscaling, R^2 improved from 0.473 to 0.475, while training R^2 remains the same. It seems changing learning rate doesn't improve that much, but maybe ok to consider.

### Therefore, epoch = 3,800 | learning rate = invscaling | solver and activation are optimized at default

### Conclusion after hyperparameter tuning
- Model having weekday and week number as inputs perform better than model having day and month number as inputs

- The first model performs best with epoch = 2,200

- The second model performs best with epoch = 3,800 and invscaling learning rate

# --- Optimization by changing train-validation-test sets

- Try 2 more random_state for 0.75 : 0.25 to see what is actuallly better model
- Try 0.85 : 0.15
- Average the performance of each try

## 0.75 : 0.25 split
### model 1 - weekday and week number

In [161]:
X = df.iloc[:, 3:5]
Y = df.iloc[:, 1:3]

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,random_state = 1)
print("size of training set :", len(Xtrain))
print("size of training set :", len(Xtest))

mlp_rg = MLPRegressor(max_iter=2200)
mlp_rg.fit(Xtrain, Ytrain)

Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

size of training set : 1359
size of training set : 453
Test R^2 Score : 0.450
Training R^2 Score : 0.483


In [162]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,random_state = 2) # change random_state to change dataset

mlp_rg = MLPRegressor(max_iter=2200)
mlp_rg.fit(Xtrain, Ytrain)

Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest))
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

Test R^2 Score : 0.499
Training R^2 Score : 0.473


In [163]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,random_state = 3) # change random_state to change dataset

mlp_rg = MLPRegressor(max_iter=2200)
mlp_rg.fit(Xtrain, Ytrain)

Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest))
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

Test R^2 Score : 0.530
Training R^2 Score : 0.499


- The split with random_state = 3 has the best performance for both testing set and training set
- The average of **test** R^2 score = 0.493
- The average of **training** R^2 score = 0.485

### model 2 - day and month

In [164]:
X2 = df.iloc[:, 5:7]
Y2 = df.iloc[:, 1:3]

Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X2,Y2,random_state = 1)
print("size of training set :", len(Xtrain2))
print("size of training set :", len(Xtest2))

mlp_rg2 = MLPRegressor(max_iter=3800, learning_rate='invscaling')
mlp_rg2.fit(Xtrain2, Ytrain2)

Ypred2 = mlp_rg2.predict(Xtest2)

print('Test R^2 Score : %.3f'%mlp_rg2.score(Xtest2, Ytest2)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg2.score(Xtrain2, Ytrain2))

size of training set : 1359
size of training set : 453
Test R^2 Score : 0.464
Training R^2 Score : 0.497


In [165]:
Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X2,Y2,random_state = 2) # change random_state to change dataset

mlp_rg2 = MLPRegressor(max_iter=3800, learning_rate='invscaling')
mlp_rg2.fit(Xtrain2, Ytrain2)

Ypred2 = mlp_rg2.predict(Xtest2)

print('Test R^2 Score : %.3f'%mlp_rg2.score(Xtest2, Ytest2)) 
print('Training R^2 Score : %.3f'%mlp_rg2.score(Xtrain2, Ytrain2))

Test R^2 Score : 0.502
Training R^2 Score : 0.475


In [166]:
Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X2,Y2,random_state = 3) # change random_state to change dataset

mlp_rg2 = MLPRegressor(max_iter=3800, learning_rate='invscaling')
mlp_rg2.fit(Xtrain2, Ytrain2)

Ypred2 = mlp_rg2.predict(Xtest2)

print('Test R^2 Score : %.3f'%mlp_rg2.score(Xtest2, Ytest2)) 
print('Training R^2 Score : %.3f'%mlp_rg2.score(Xtrain2, Ytrain2))

Test R^2 Score : 0.506
Training R^2 Score : 0.495


- The split with random_state = 3 has the best performance for both testing set and training set as well as for both models
- The average of **test** R^2 score = 0.491
- The average of **training** R^2 score = 0.489

**Conclusion**

- Both models have slightly different performance level which model 1 that uses weekday and week number as inputs is better

## 0.85 : 0.15 split -- model 1
### Optimize the epoch size

In [167]:
X = df.iloc[:, 3:5]
Y = df.iloc[:, 1:3]

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,random_state = 1, test_size=0.15) # change test set size
print("size of training set :", len(Xtrain))
print("size of training set :", len(Xtest))

parameter_space = {
    'max_iter': np.arange(2000,4001,200)
}

clf = GridSearchCV(mlp_rg, parameter_space, n_jobs=-1, cv=5)
clf.fit(Xtrain, Ytrain)

size of training set : 1540
size of training set : 272


GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=2200), n_jobs=-1,
             param_grid={'max_iter': array([2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000])})

In [168]:
print(clf.best_params_)

{'max_iter': 2400}


So, we will go for epoch = 2400 for this proportion of split

In [169]:
mlp_rg = MLPRegressor(max_iter=2400)
mlp_rg.fit(Xtrain, Ytrain)

Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

Test R^2 Score : 0.428
Training R^2 Score : 0.489


In [170]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,random_state = 2) # change random_state to change dataset

mlp_rg = MLPRegressor(max_iter=2400)
mlp_rg.fit(Xtrain, Ytrain)

Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest))
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

Test R^2 Score : 0.516
Training R^2 Score : 0.486


In [171]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,random_state = 3) # change random_state to change dataset

mlp_rg = MLPRegressor(max_iter=2400)
mlp_rg.fit(Xtrain, Ytrain)

Ypred = mlp_rg.predict(Xtest)

print('Test R^2 Score : %.3f'%mlp_rg.score(Xtest, Ytest))
print('Training R^2 Score : %.3f'%mlp_rg.score(Xtrain, Ytrain))

Test R^2 Score : 0.509
Training R^2 Score : 0.479


**Conclusion**

- The split with random_state = 2,3 has the best performance for both testing set and training set as well as for both models
- The split with random_state = 1 has big drop of test R^2 score compared to other random_state
- The average of **test** R^2 score = 0.484
- The average of **training** R^2 score = 0.485

# Conclusion
- Model that uses weekday and week number to predict does the better job, more with train-test split of 0.75 : 0.25 and more with epoch = 2,200 with average R^2 = 0.493